<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#循环神经网络-(recurrent-neural-network,-RNN)" data-toc-modified-id="循环神经网络-(recurrent-neural-network,-RNN)-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>循环神经网络 (recurrent neural network, RNN)</a></span></li><li><span><a href="#长短时记忆网络-(LTSM)" data-toc-modified-id="长短时记忆网络-(LTSM)-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>长短时记忆网络 (LTSM)</a></span></li><li><span><a href="#循环神经网络的变种" data-toc-modified-id="循环神经网络的变种-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>循环神经网络的变种</a></span><ul class="toc-item"><li><span><a href="#双向循环神经网络-(bidirectional-RNN)" data-toc-modified-id="双向循环神经网络-(bidirectional-RNN)-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>双向循环神经网络 (bidirectional RNN)</a></span></li><li><span><a href="#深层循环神经网络-(deepRNN)" data-toc-modified-id="深层循环神经网络-(deepRNN)-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>深层循环神经网络 (deepRNN)</a></span></li><li><span><a href="#循环神经网络的dropout" data-toc-modified-id="循环神经网络的dropout-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>循环神经网络的dropout</a></span></li></ul></li><li><span><a href="#利用RNN训练手写数字MNIST数据集" data-toc-modified-id="利用RNN训练手写数字MNIST数据集-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>利用RNN训练手写数字MNIST数据集</a></span><ul class="toc-item"><li><span><a href="#tf.nn.dynamic_rnn-函数说明" data-toc-modified-id="tf.nn.dynamic_rnn-函数说明-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>tf.nn.dynamic_rnn 函数说明</a></span></li></ul></li></ul></div>

## 循环神经网络 (recurrent neural network, RNN)
[Link1](https://www.imooc.com/article/23821)


> 循环神经网络的主要用途是处理和预测`序列数据`。循环神经网络的隐藏层之间的结点是有连接的，隐藏层的输入不仅包括输入层的输出，还包括上一时刻隐藏层的输出。

> 循环神经网络会对于每一个时刻的输入结合当前模型的状态给出一个输出。循环神经网络在每一个时刻会有一个输入X(t)，然后根据循环神经网络当前的状态A(t)提供一个输出H(t)；而循环神经网络当前的状态A(t)是根据上一时刻的状态A(t-1)和当前的输入X(t)共同决定的。

> 对于一个序列数据，可以将这个序列上不同时刻的数据依次传入循环神经网络的输入层，而输出可以是对序列中下一个时刻的预测，也可以是对当前时刻信息的处理结果。

> 在循环神经网络中，循环体网络结构中的参数`在不同时刻是共享的`。

> 循环神经网络的总损失为所有时刻(或者部分时刻)上的损失函数的总和。

![RNN前向传播计算示意图](illustration/RNN前向传播计算示意图.png)

## 长短时记忆网络 (LTSM)

> 循环神经网络工作的关键点是使用历史的信息来帮助当前的决策；但同时也带来了更大的技术挑战 -- 长期依赖(long-term dependencies)

> LSTM的提出就是为了解决长期依赖问题。LSTM是一种拥有三个"门"结构的特殊网络结构。(所谓"门"的结构就是一个使用sigmoid神经网络和和一个按位做乘法的操作，这两个操作合在一起就是一个"门"结构，之所以该结构叫做"门"是因为使用sigmoid作为激活函数的全连接神经网络层会输出一个0到1之间的数值，描述当前输入有多少信息量可以通过这个结构)

> "遗忘门"会根据当前的输入`x(t)`, 上一时刻状态`c(t-1)`和上一时刻输出`h(t-1)`共同决定哪一部分记忆需要被遗忘

> "输入门"会根据`x(t)`、`c(t-1)`和`h(t-1)`决定哪些部分将进入当前时刻的状态`c(t)`

> "输出门"会根据最新的状态`c(t)`、上一时刻的输出`h(t-1)`和当前的输入`x(t)`来决定该时刻的输出`h(t)`


![LSTM单元结构示意图](illustration/LSTM单元结构示意图.png)


In [ ]:
# 定义一个LSTM结构, 并指定隐藏层的大小
lstm = tf.contrib.rnn.BasicLSTMCell(lstm_hidden_size)
'''
__init__(
    num_units,
    forget_bias=1.0,
    state_is_tuple=True,
    activation=None,
    reuse=None,
    name=None,
    dtype=None
)
'''

# 将LSTM中的状态初始化为全0数组. 和其他神经网络类似, 在优化神经网络时, 
# 每次也会使用一个batch的训练样本, 以下代码中, batch_size给出了一个batch大小
# BasicLSTMCell类提供了zero_state函数来生成全0的初始状态
state = lstm.zero_state(batch_size, tf.float32)
'''
zero_state(batch_size, dtype)
Args:
    batch_size:  int, float, or unit Tensor representing the batch size.
    dtype:  the data type to use for the state.
Return:
    zero-filled state tensor(s).    
'''

# 虽然理论上循环神经网络可以处理任意长度的序列, 但是在训练时为了避免梯度消散的问题,
# 通常会规定一个最大的序列长度, 以下代码中, 用 num_steps 来表示这个长度
for i in range(num_steps):
    # 在第一个时刻声明LSTM结构中使用的变量
    # 在之后的时刻都需要复用之前定义好的变量
    if i > 0:
        tf.get_variable_scope().reuse_variables()
    # 每一步处理时间序列中的一个时刻。将当前输入(current_input)和前一时刻状态(state)
    # 传入定义的LSTM结构可以得到当前LSTM结构的输出lstm_output和更新后的状态state
    lstm_output, state = lstm(current_input, state)
    # 将当前时刻LSTM结构的输出传入一个全连接层得到最后的输出
    final_output = fully_connected(lstm_output)
    

## 循环神经网络的变种

### 双向循环神经网络 (bidirectional RNN)

双向循环神经网络是由两个循环神经网络上下叠加在一起组成的。输出由这两个循环神经网络的状态共同决定。  
![双向循环神经网络结构示意图](illustration/双向循环神经网络结构示意图.png)  

> 双向循环神经网络的主体结构就是两个单向循环神经网络的结合。在每一个时刻`t`, 输入会同时提供给这两个方向相反的循环神经网络, 而输出则是由这两个单向循环神经网络共同决定。双向循环神经网络的前向传播过程和单向的循环神经网络十分类似

### 深层循环神经网络 (deepRNN)
深层循环神经网络在每一个时刻上将循环体结构复制了多次，每一层的循环体中参数是一致的，而不同层中的参数可以不同   
![深层循环神经网络结构示意图](illustration/深层循环神经网络结构示意图.png)

### 循环神经网络的dropout
在循环神经网络中使用`dropout`可以使网络更加健壮；区别于卷积神经网络只在最后的全连接层中使用`dropout`, 循环神经网络一般只在不同层循环体结构之间使用`dropout`, 而不在同一层的循环体结构之间使用。也就是说从时刻`t-1`传递到时刻`t`时，循环神经网络不会进行状态的`dropout`；而在同一时刻`t`中，不同层循环体之间会使用`dropout`  
![深层循环神经网络使用dropout示意图](illustration/深层循环神经网络使用dropout示意图.png)

> 图中实线箭头表示不使用`dropout`，虚线箭头表示使用`dropout`

## 利用RNN训练手写数字MNIST数据集
[Link1](https://blog.csdn.net/omnispace/article/details/78415100)

In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
tf.set_random_seed(1)   # set random seed
 
# 导入数据
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)
 
# hyperparameters
lr = 0.001                         # learning rate
training_iters = 100000     # train step 上限
batch_size = 128            
n_inputs = 28                  # MNIST data input (img shape: 28*28)
n_steps = 28                   # time steps
n_hidden_units = 128       # neurons in hidden layer
n_classes = 10                # MNIST classes (0-9 digits)

# x y placeholder
x = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.float32, [None, n_classes])
 
# 对 weights biases 初始值的定义
weights = {
    # shape (28, 128)
    'in': tf.Variable(tf.random_normal([n_inputs, n_hidden_units])),
    # shape (128, 10)
    'out': tf.Variable(tf.random_normal([n_hidden_units, n_classes]))
}
biases = {
    # shape (128, )
    'in': tf.Variable(tf.constant(0.1, shape=[n_hidden_units, ])),
    # shape (10, )
    'out': tf.Variable(tf.constant(0.1, shape=[n_classes, ]))
}

'''
  RNN 主体总共有3部分组成(input_layer[输入隐藏层],  rnn_cell[rnn网络层],  output_layer[输出隐藏层])
'''
def RNN(X, weights, biases):
    # 定义 input_layer 层...
    # 原始的 X 是 3 维数据, 我们需要把它变成 2 维数据才能使用 weights 的矩阵乘法
    # X ==> (128 batches * 28 steps, 28 inputs)
    X = tf.reshape(X, [-1, n_inputs])
    # X_in = W*X + b
    X_in = tf.matmul(X, weights['in']) + biases['in']
    # X_in ==> (128 batches, 28 steps, 128 hidden) 换回3维
    X_in = tf.reshape(X_in, [-1, n_steps, n_hidden_units])

    # 定义 rnn_cell 层...
    # 使用 basic LSTM Cell.
    # 对于 lstm 来说, state可被分为(c_state, h_state).
    lstm_cell = tf.contrib.rnn.BasicLSTMCell(n_hidden_units, forget_bias=1.0, state_is_tuple=True)
    # 初始化全零 state
    init_state = lstm_cell.zero_state(batch_size, dtype=tf.float32) 
    # 如果 inputs 为 (batches, steps, inputs) ==> time_major=False;
    # 如果 inputs 为 (steps, batches, inputs) ==> time_major=True;
    outputs, final_state = tf.nn.dynamic_rnn(lstm_cell, X_in, initial_state=init_state, time_major=False)

    # 定义 output_layer 层...
    # 由于该例的特殊性, 有两种方法来计算 output_layer 输出
    # 方式一: 直接调用final_state 中的 h_state ( 通过final_state[1]获取) 来进行运算:
    results = tf.matmul(final_state[1], weights['out']) + biases['out']  
    # 方式二: 调用最后一个 outputs (在这个例子中, 和上面的final_state[1]是一样的):
    # 由于time_major=False,  故outputs 的形状为 [batch_size, max_time, cell.output_size]
    # 把 outputs 变成 列表 [(batch, outputs)..] * steps
    outputs = tf.unstack(tf.transpose(outputs, [1,0,2]))
    results = tf.matmul(outputs[-1], weights['out']) + biases['out']    #选取最后一个 output

return results

# 定义 cost 和 train_op:
pred = RNN(x, weights, biases)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y))
train_op = tf.train.AdamOptimizer(lr).minimize(cost)

# 训练 RNN
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
init = tf.global_variables_initializer()
 
with tf.Session() as sess:
    sess.run(init)
    step = 0
    while step * batch_size < training_iters:
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = batch_xs.reshape([batch_size, n_steps, n_inputs])
        sess.run([train_op], feed_dict={
            x: batch_xs,
            y: batch_ys,
        })
        if step % 20 == 0:
            print(sess.run(accuracy, feed_dict={
            x: batch_xs,
            y: batch_ys,
        }))
        step += 1

### tf.nn.dynamic_rnn 函数说明

```python
tf.nn.dynamic_rnn(
    cell,
    inputs,
    sequence_length=None,
    initial_state=None,
    dtype=None,
    parallel_iterations=None,
    swap_memory=False,
    time_major=False,
    scope=None
)
```

Returns:   A pair (outputs, state)
[Link1](https://blog.csdn.net/u010960155/article/details/81707498)

outputs 是一个tensor  
> 如果time_major==True，outputs形状为 [max_time, batch_size, cell.output_size]（要求rnn输入与rnn输出形状保持一致）     

> 如果time_major==False(默认)，outputs形状为 [batch_size, max_time, cell.output_size]
   

state 是一个tensor  
> 如果cell为LSTM, state形状为[2，batch_size, cell.output_size]; 每个cell会有两个输出: `C(t)` 和 `h(t)`，上面这个图是输出`C(t)`，代表哪些信息应该被记住哪些应该被遗忘；下面这个图是输出`h(t)`，代表这个cell的最终输出，LSTM的state是由`C(t)` 和 `h(t)` 组成的

> 如果cell为GRU, state形状为[batch_size, cell.output_size];  state就只有一个了，原因是GRU将`C(t)` 和 `h(t)` 进行了简化，将其合并成了`h(t)`，如下图所示，GRU将遗忘门和输入门合并成了更新门，另外cell不在有细胞状态cell state，只有hidden state

output 与 state 的关系
> 如果cell为LSTM，那 state是个tuple，分别代表`C(t)` 和 `h(t)` ，其中 `h(t)` 与outputs中的对应的最后一个时刻的输出相等，假设state形状为[2，batch_size, cell.output_size]，outputs形状为 [batch_size, max_time, cell.output_size]，那么`state[1, batch_size, : ]` == `outputs[batch_size, -1, : ]`

> 如果cell为GRU，同上，state其实就是 `h(t)`，`state` == `outputs[-1]`


```python
# create a BasicRNNCell
rnn_cell = tf.nn.rnn_cell.BasicRNNCell(hidden_size)

# 'outputs' is a tensor of shape [batch_size, max_time, cell_state_size]

# defining initial state
initial_state = rnn_cell.zero_state(batch_size, dtype=tf.float32)

# 'state' is a tensor of shape [batch_size, cell_state_size]
outputs, state = tf.nn.dynamic_rnn(rnn_cell, input_data,
                                   initial_state=initial_state,
                                   dtype=tf.float32)
```

```python
# create 2 LSTMCells
rnn_layers = [tf.nn.rnn_cell.LSTMCell(size) for size in [128, 256]]

# create a RNN cell composed sequentially of a number of RNNCells
multi_rnn_cell = tf.nn.rnn_cell.MultiRNNCell(rnn_layers)

# 'outputs' is a tensor of shape [batch_size, max_time, 256]
# 'state' is a N-tuple where N is the number of LSTMCells containing a
# tf.contrib.rnn.LSTMStateTuple for each cell
outputs, state = tf.nn.dynamic_rnn(cell=multi_rnn_cell,
                                   inputs=data,
                                   dtype=tf.float32)
```